I'll be using CNN articles as dataset. These articles are stored in multiple files along with their highlights. These highlight can be used to gauge the summary output. 

In [104]:
from tqdm.notebook import tqdm 

Let's try reading the file content 

In [105]:
def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    return text

text = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')
#text.split("\n")



It seems the we need to do some cleaning before we start working on the file.

In [106]:
import re
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
en_stopwords = stopwords.words('english')

def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    # cnn_indx = text.find("(CNN)")
    # if cnn_indx >=0:
    #     text = text[cnn_indx+len("(CNN)"):]
        
    #text = re.sub("\n"," ",text).lower()

    #text.split("\n")
    # cleaned = list()
    # table = str.maketrans('','',string.punctuation)

    # for line in text:
       
    #     line = line.split(" ")

    #     line = [w.translate(table) for w in line]
        
    #     line = [w for w in line if w.isalpha()]
    #     cleaned.append(' '.join(line))

    # text = [sent for sent in cleaned if len(sent)>0]

    return text

article = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')
dmarticle = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/dailymail/stories/0a00a9aebcb754c51534867cf1db2335dcb76884.story')
# t = article.split("\n")
# cleaned = list()
# table = str.maketrans('','',string.punctuation)
# for w in t:
#     if len(w)>0:
#         w = w.split(" ")
#         w = [word.translate(table) for word in w]
#         w = [word for word in w if word.isalpha()]
#         cleaned.append(" ".join(w))
# print(cleaned)
print(dmarticle)


Manchester United's Spaniards may have struggled to gel on the pitch, but they have been building up a close relationship off it.

Juan Mata, Ander Herrera and David de Gea have become good friends and enjoyed lunch together in Hale Village, Cheshire on Tuesday afternoon.

The Spanish trio were all named in Louis van Gaal's starting line-up for the opening day fixture against Swansea, although they were unable to prevent a disappointing 2-1 defeat bySwansea.

VIDEO Scroll down to watch Mata challenge compatriot De Gea to Corner Kick Challenge 

Meet and greet: Juan Mata (left) has been helping fellow Spaniard Ander Herrera (right) settle in

Join the club: Manchester United keeper David de Gea and Herrera pictured together on Tuesday

Three Amigos: The Spaniards stroll down the high street as they enjoy an afternoon in Hale Village

Van Gaal has brought another Spanish speaker to Old Trafford, with Argentina defender Marcos Rojo describing it as a 'dream' to play for the Old Trafford c

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rohankilledar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now we need to split the article and the hightlights

In [107]:
def split_text(article):
    indx = article.index('@highlight')
    story = article[:indx]
    highlight = article[indx:].split('@highlight')

    highlight = ". ".join([h.strip() for h in highlight if len(h)>0])
    return story,highlight

st,hi = split_text(dmarticle)
print(st)
print(hi)

Manchester United's Spaniards may have struggled to gel on the pitch, but they have been building up a close relationship off it.

Juan Mata, Ander Herrera and David de Gea have become good friends and enjoyed lunch together in Hale Village, Cheshire on Tuesday afternoon.

The Spanish trio were all named in Louis van Gaal's starting line-up for the opening day fixture against Swansea, although they were unable to prevent a disappointing 2-1 defeat bySwansea.

VIDEO Scroll down to watch Mata challenge compatriot De Gea to Corner Kick Challenge 

Meet and greet: Juan Mata (left) has been helping fellow Spaniard Ander Herrera (right) settle in

Join the club: Manchester United keeper David de Gea and Herrera pictured together on Tuesday

Three Amigos: The Spaniards stroll down the high street as they enjoy an afternoon in Hale Village

Van Gaal has brought another Spanish speaker to Old Trafford, with Argentina defender Marcos Rojo describing it as a 'dream' to play for the Old Trafford c

one of the dataset file seems to be usable now. Need to load all of the stories together using these given functions

In [108]:
from os import listdir

def read_all(folder):
    dataset = list()

    for file in tqdm(listdir(folder)):
        filename = folder + '/' + file
        article = read_file(filename)
        story,highlight = split_text(article)

        dataset.append({'story':story, 'highlight':highlight})
    
    return dataset

folder = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories"
daily_mail = '/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/dailymail/stories'

dataset = read_all(folder)
daily_mail_dataset = read_all(daily_mail)
dataset.extend(daily_mail_dataset)

print("total stories in dataset: "+ str(len(dataset)))

  0%|          | 0/92579 [00:00<?, ?it/s]

  0%|          | 0/219506 [00:00<?, ?it/s]

total stories in dataset: 312085


In [109]:
example = dataset[1]['highlight']
example


'Zimmerman posts $5,000 bail; he was accused of throwing a bottle at a girlfriend. "He hasn\'t been very lucky with the ladies," attorney says of Zimmerman. He became a national figure after being charged, then acquitted in Trayvon Martin\'s death'

Data Cleaning

In [110]:
import re
def clean_data(data):
    cleaned = list()
    table = str.maketrans('', '', string.punctuation)
    for line in data:
        #line = re.split(r"(CNN)+ --",line)

        dash_indx = line.find("--")
        if dash_indx >=0:
            line = line[dash_indx+len("--"):]

        #line = line.trim()
        # tokenize on white space
        line = line.split()
        # convert to lower case
        line = [word.lower() for word in line]
        # remove punctuation from each token
        line = [w.translate(table) for w in line]
        # remove tokens with numbers in them
        line = [word for word in line if word.isalpha()]
        # store as string
        cleaned.append(' '.join(line))
	# remove empty strings
    cleaned = [c for c in cleaned if len(c) > 0]
    return cleaned


#dataset[0]['story'] = clean_data(dataset[0]['story'].split('\n'))
#dataset[0]['highlight'] = clean_data(dataset[0]['highlight'])
#     #text = re.sub("\n"," ",text).lower()
# cleaned_dataset = dataset.copy()
for i in tqdm(range(len(dataset))):

    dataset[i]['story']= clean_data(dataset[i]['story'].split('\n'))#.lstrip("[\'").rstrip("\']")
    dataset[i]['highlight'] = clean_data(dataset[i]['highlight'].split('\n'))#.lstrip("[\'").rstrip("\']")
    

  0%|          | 0/312085 [00:00<?, ?it/s]

In [111]:
dataset[100000]['highlight']

['delta plane circled airport for minutes following the blunder the federal aviation administration is investigating the incident air traffic controller told pilot im kidding but plane had already ascended']

Saving the dataset

In [113]:
import csv
csv_columns = ['story','highlight']
csv_file = "data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dataset:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [114]:
import pandas as pd

dataset_pd = pd.read_csv("data.csv")

In [115]:
#dataset_pd['highlight'][0].lstrip("[\'").rstrip("\']")
dataset_pd['story'][0].lstrip("[\'").rstrip("\']")

"at the start of a big week for the higgs boson the most soughtafter particle in all of physics scientists in illinois said monday that they had crept closer to proving that the particle exists but had been unable to reach a definitive conclusion', 'the scientists outlined their final analysis based on more than years of research and trillion particle collisions using the us department of energys fermilab tevatron collider near batavia illinois whose budgetary woes shut it down last year', 'what is the higgs boson and why is it important', 'their announcement came two days before researchers at the large hadron collider under the alps are due to unveil their latest results at an eagerly awaited seminar at the cern particle physics laboratory in geneva switzerland', 'our data strongly point toward the existence of the higgs boson rob roser a spokesman for one of two independent experiments at the tevatron said in a statement but it will take results from the experiments at the large had

In [124]:
dataset_pd['highlight'] = dataset_pd['highlight'].apply(lambda x:  'sostok ' + x.lstrip("[\'").rstrip("\']") + ' eostok')
dataset_pd['story'] = dataset_pd['story'].apply(lambda x: x.lstrip("[\'").rstrip("\']") )


In [138]:
import pickle
output_file = open('dataset.pkl','wb')
pickle.dump(dataset_pd, output_file)
output_file.close()

In [126]:
#dataset_pd.drop('hightlight1', axis=1, inplace=True)
dataset_pd['word_count_text'] = dataset_pd['story'].apply(lambda x: len(str(x).split()))

In [134]:
import numpy as np
for i in range(0,100,10) :
    var =dataset_pd["word_count_text"]. values
    var = np.sort(var,axis = None)
    print (" {} percentile value is {}".format (i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

 0 percentile value is 0
 10 percentile value is 293
 20 percentile value is 379
 30 percentile value is 451
 40 percentile value is 521
 50 percentile value is 592
 60 percentile value is 672
 70 percentile value is 766
 80 percentile value is 891
 90 percentile value is 1096
100 percentile value is  2094


In [135]:
for i in range (90,100) :
    var =dataset_pd["word_count_text"].values
    var = np.sort(var, axis = None)
    print("{} percentile value is {}".format(i, var[int (len(var)*(float (i)/100))]))
print ("100 percentile value is ", var[-1])

90 percentile value is 1096
91 percentile value is 1126
92 percentile value is 1159
93 percentile value is 1195
94 percentile value is 1236
95 percentile value is 1285
96 percentile value is 1340
97 percentile value is 1403
98 percentile value is 1482
99 percentile value is 1579
100 percentile value is  2094
